# 워드 클라우드(단어구름) 

In [1]:
import numpy as np
import random
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
from palettable.colorbrewer.qualitative import Dark2_8
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# color_func 함수, Dark2_8이라는 pallette를 이용하여 색을 만들어냄.
def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return tuple(Dark2_8.colors[random.randint(0,7)])

# 한글 폰트 지정
font = "NanumPen"
font_path = "%s.ttf" % font

# 아이콘 지정
icon = "house" 
icon_path = "%s.png" % icon

### 데이터 입력

In [4]:
# 연설문1
# 아래 KoreaSummit_Moon.txt를 본인이 저장한 파일 이름으로 바꾸세요.
#-*- coding: utf-8 -*-
f = open("badreview.txt", 'r', encoding='euc-kr')
message_of_guests = f.read()
print(message_of_guests)
f.close()

위치설명과 소통은 아주 잘되었어요. 하지만 화장실과 침대 청결도가 별로였고 다른 모르는 두분이랑 같은 집을 써야하는게 별로 좋지 않았어요


Good 처음 오는 사람 누구나 쉽게 이용할 수 있도록 그리고 제가 시간이 좀 늦고 특이사항이 있었는데 그에준비하셔서 미리 쪽지로 안내도 해주셔서 좋았습니다 Bad 환절기라 보일러를 잘 확인해야는데 이 부분은 잘 챙겨주시면 좋을 것 같습니다 그리고 방 사진이 달라서 살짝 놀랐어요 방에도 포샵 기능이?ㅎ


더럽고 좁고 사진빨이에요. 오지 말도록 하세요。


여행 일정에 공백이생겨 적은 가격으로 하루 묵을 목적으로 숙소를 잡았어요. 가격은 저렴해서 좋았지만 솔직히 아쉬움이 많이 남는 숙소였어요. 무엇보다 큰 창에 비해서 너무 작은커튼과 화장실의 냄새는 가장 아쉬웠던 부분인거 같네요.


홍대 주변이라 편의시설이 좋음 보일러를 직접 작동할 수 있어서 춥지 않음(방은 약간 찬바람이 들어옴) 8명이 숙박하기엔 이불/수건 등 제공해주는 물품들이 부족함 전체적으로 이전 사람들이 사용했던 흔적들이 많았음 사전에 연락도 없이 다음날 숙박예정인 외국인들이 짐을 놓겠다고 무작정 들어옴


빔프로젝트있어서 갔는데 유에스비없으면 못써서 꼭 챙겨가시구요. 티비도 없고 휴지도 조금 부족하고 인원많으시면 수건도 부족하실 수도 있어요! 가격대비 나쁘지 않았지만 그래도 저흰 좀 아쉬웠던 것 같아요!


우선 위치면에서는 홍대입구 근처여서 놀다가 들어가기 좋았습니다~ 하지만 위치를 제외하고 숙소의 용도면에서는.. 주방기구에 기본적인 가위 등도 없어서 음식을 해먹기에는 무리같네요ㅎㅎ 이 점 제외하고 이동성 고려하신다 하면 좋은 것 같습니다


호스트분은 정말 친절하시고 연락도 잘 되어서 좋았어요. 원래 예약한 방이 수도공사로 다른 집을 전체로 쓰실수 있게해주셔서 좋았는데, 생각보다 집이 별로였습니다. ㅠㅠ 일단 작은방 창문이 밖에서도 열릴수 있게 되어있어서 불안했고요.. (위치가 어둡고 반지하였습니다;) 비가 와서 그런지 정말 집이 추웠고.. 화장실이 낡

In [18]:
# 연설문2
f = open("goodreview.txt", 'r', encoding='euc-kr')
message_of_guests2 = f.read()
print(message_of_guests2)
f.close()

UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x89 in position 1055: illegal multibyte sequence

In [5]:
# 연설문 1번 전처리
message_of_guests = message_of_guests.split('\n\n')
message_of_guests

['위치설명과 소통은 아주 잘되었어요. 하지만 화장실과 침대 청결도가 별로였고 다른 모르는 두분이랑 같은 집을 써야하는게 별로 좋지 않았어요',
 '\nGood 처음 오는 사람 누구나 쉽게 이용할 수 있도록 그리고 제가 시간이 좀 늦고 특이사항이 있었는데 그에준비하셔서 미리 쪽지로 안내도 해주셔서 좋았습니다 Bad 환절기라 보일러를 잘 확인해야는데 이 부분은 잘 챙겨주시면 좋을 것 같습니다 그리고 방 사진이 달라서 살짝 놀랐어요 방에도 포샵 기능이?ㅎ',
 '\n더럽고 좁고 사진빨이에요. 오지 말도록 하세요。',
 '\n여행 일정에 공백이생겨 적은 가격으로 하루 묵을 목적으로 숙소를 잡았어요. 가격은 저렴해서 좋았지만 솔직히 아쉬움이 많이 남는 숙소였어요. 무엇보다 큰 창에 비해서 너무 작은커튼과 화장실의 냄새는 가장 아쉬웠던 부분인거 같네요.',
 '\n홍대 주변이라 편의시설이 좋음 보일러를 직접 작동할 수 있어서 춥지 않음(방은 약간 찬바람이 들어옴) 8명이 숙박하기엔 이불/수건 등 제공해주는 물품들이 부족함 전체적으로 이전 사람들이 사용했던 흔적들이 많았음 사전에 연락도 없이 다음날 숙박예정인 외국인들이 짐을 놓겠다고 무작정 들어옴',
 '\n빔프로젝트있어서 갔는데 유에스비없으면 못써서 꼭 챙겨가시구요. 티비도 없고 휴지도 조금 부족하고 인원많으시면 수건도 부족하실 수도 있어요! 가격대비 나쁘지 않았지만 그래도 저흰 좀 아쉬웠던 것 같아요!',
 '\n우선 위치면에서는 홍대입구 근처여서 놀다가 들어가기 좋았습니다~ 하지만 위치를 제외하고 숙소의 용도면에서는.. 주방기구에 기본적인 가위 등도 없어서 음식을 해먹기에는 무리같네요ㅎㅎ 이 점 제외하고 이동성 고려하신다 하면 좋은 것 같습니다',
 '\n호스트분은 정말 친절하시고 연락도 잘 되어서 좋았어요. 원래 예약한 방이 수도공사로 다른 집을 전체로 쓰실수 있게해주셔서 좋았는데, 생각보다 집이 별로였습니다. ㅠㅠ 일단 작은방 창문이 밖에서도 열릴수 있게 되어있어서 불안했고요.. (위치가 어둡고 반지하였습니다;

In [7]:
# 연설문 1번 빈도 계산
# except 1-letter word
## min_df: integer(frequency), float(ratio)
tf_vectorizer = CountVectorizer(min_df = 0.001, token_pattern=r'\w{2,}')

tf_corpus1 = tf_vectorizer.fit(message_of_guests)
tf_bow1 = tf_vectorizer.fit_transform(message_of_guests)
tf_bow1

<60x1527 sparse matrix of type '<class 'numpy.int64'>'
	with 2220 stored elements in Compressed Sparse Row format>

In [8]:
# 연설문 1 
# check vocabulary in TDM
print(len(tf_corpus1.vocabulary_))
print(len(tf_corpus1.get_feature_names()))
tf_vectorizer.get_feature_names()

1527
1527


['1030',
 '10분정도',
 '11시',
 '11시에는',
 '12',
 '12시',
 '12시에',
 '1박',
 '1시간을',
 '1층이긴',
 '2개',
 '2개있는',
 '2명이서',
 '2박',
 '2박만',
 '2주간',
 '30',
 '3분',
 '3시간',
 '3시이며',
 '4시간',
 '4층',
 '5시간',
 '5층같은',
 '6도의',
 '6분거리라는점입니다',
 '7시',
 '8명이',
 '8시',
 'airbnb',
 'bad',
 'good',
 'mom님',
 'mori',
 'ㅋㅋ',
 'ㅋㅋㅋ',
 'ㅎㅎ',
 'ㅎㅎㅋㅋ',
 'ㅎㅎ너무',
 'ㅠㅠ',
 '가격값하는',
 '가격대비',
 '가격에',
 '가격으로',
 '가격은',
 '가격이',
 '가기엔',
 '가까우며',
 '가까운',
 '가까운편이고',
 '가까웠는데',
 '가까웠습니다',
 '가까이',
 '가는',
 '가는법',
 '가도',
 '가로등이',
 '가야할듯',
 '가요',
 '가위',
 '가장',
 '가져',
 '가져왔다',
 '가지고',
 '가질러갔어요',
 '간다면',
 '갈게요',
 '갈아',
 '감사합니다',
 '감사해요',
 '갔는데',
 '갔더니',
 '갔습니다',
 '강남역',
 '강남이랑',
 '갖춰진',
 '같네요',
 '같습니다',
 '같아요',
 '같은',
 '같이',
 '개선되어야',
 '개인의',
 '거리',
 '거리로',
 '거리와',
 '거부감이',
 '거실',
 '거의',
 '거주자가',
 '거주하고',
 '건들거려서',
 '건물',
 '건물과',
 '건물입구까지',
 '걸려서',
 '걸렸으므로',
 '걸리고',
 '걸린것',
 '걸립니다',
 '걸어다닐',
 '걸으면',
 '걸을때마다',
 '검사',
 '검사가',
 '겁에',
 '것들은',
 '것을',
 '것이',
 '것이고',
 '것이라고',
 '것인지',
 '게스트',
 '게스트들에게',
 '게스트분들께는',
 '겨울이라',
 '결국',
 '결론',
 '결정했습니다',

In [9]:
# 연설문 1
# frequency count
tf_word_sum1 = tf_bow1.toarray().sum(axis=0)
tf_word_name1 = tf_corpus1.get_feature_names()
tf_word_dict1 = {}

for i in range(len(tf_word_sum1)):
    tf_word_dict1[tf_word_name1[i]] = tf_word_sum1[i]
    
print(tf_corpus1.get_feature_names())
tf_word_sum1

['1030', '10분정도', '11시', '11시에는', '12', '12시', '12시에', '1박', '1시간을', '1층이긴', '2개', '2개있는', '2명이서', '2박', '2박만', '2주간', '30', '3분', '3시간', '3시이며', '4시간', '4층', '5시간', '5층같은', '6도의', '6분거리라는점입니다', '7시', '8명이', '8시', 'airbnb', 'bad', 'good', 'mom님', 'mori', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅎㅎ', 'ㅎㅎㅋㅋ', 'ㅎㅎ너무', 'ㅠㅠ', '가격값하는', '가격대비', '가격에', '가격으로', '가격은', '가격이', '가기엔', '가까우며', '가까운', '가까운편이고', '가까웠는데', '가까웠습니다', '가까이', '가는', '가는법', '가도', '가로등이', '가야할듯', '가요', '가위', '가장', '가져', '가져왔다', '가지고', '가질러갔어요', '간다면', '갈게요', '갈아', '감사합니다', '감사해요', '갔는데', '갔더니', '갔습니다', '강남역', '강남이랑', '갖춰진', '같네요', '같습니다', '같아요', '같은', '같이', '개선되어야', '개인의', '거리', '거리로', '거리와', '거부감이', '거실', '거의', '거주자가', '거주하고', '건들거려서', '건물', '건물과', '건물입구까지', '걸려서', '걸렸으므로', '걸리고', '걸린것', '걸립니다', '걸어다닐', '걸으면', '걸을때마다', '검사', '검사가', '겁에', '것들은', '것을', '것이', '것이고', '것이라고', '것인지', '게스트', '게스트들에게', '게스트분들께는', '겨울이라', '결국', '결론', '결정했습니다', '경우', '경우가', '경험', '경험이', '경험인데', '계단은', '계신지', '고려하신다', '고려해야합니다', '고시룸', '고양이가', '고장나', '고장난', '고정되어', '고지받지', '곳곳에

array([1, 1, 2, ..., 3, 1, 2], dtype=int64)

In [11]:
# 연설문 1
# 단어별 글자수 확인
tf_word_dict1

{'1030': 1,
 '10분정도': 1,
 '11시': 2,
 '11시에는': 1,
 '12': 1,
 '12시': 1,
 '12시에': 1,
 '1박': 1,
 '1시간을': 1,
 '1층이긴': 1,
 '2개': 2,
 '2개있는': 2,
 '2명이서': 1,
 '2박': 1,
 '2박만': 1,
 '2주간': 1,
 '30': 2,
 '3분': 1,
 '3시간': 1,
 '3시이며': 1,
 '4시간': 2,
 '4층': 1,
 '5시간': 1,
 '5층같은': 1,
 '6도의': 1,
 '6분거리라는점입니다': 1,
 '7시': 2,
 '8명이': 1,
 '8시': 2,
 'airbnb': 1,
 'bad': 1,
 'good': 1,
 'mom님': 1,
 'mori': 1,
 'ㅋㅋ': 2,
 'ㅋㅋㅋ': 2,
 'ㅎㅎ': 1,
 'ㅎㅎㅋㅋ': 2,
 'ㅎㅎ너무': 1,
 'ㅠㅠ': 2,
 '가격값하는': 1,
 '가격대비': 2,
 '가격에': 3,
 '가격으로': 3,
 '가격은': 1,
 '가격이': 2,
 '가기엔': 2,
 '가까우며': 2,
 '가까운': 2,
 '가까운편이고': 1,
 '가까웠는데': 1,
 '가까웠습니다': 2,
 '가까이': 2,
 '가는': 2,
 '가는법': 1,
 '가도': 1,
 '가로등이': 2,
 '가야할듯': 2,
 '가요': 1,
 '가위': 1,
 '가장': 3,
 '가져': 1,
 '가져왔다': 1,
 '가지고': 1,
 '가질러갔어요': 1,
 '간다면': 1,
 '갈게요': 1,
 '갈아': 1,
 '감사합니다': 1,
 '감사해요': 1,
 '갔는데': 3,
 '갔더니': 1,
 '갔습니다': 1,
 '강남역': 3,
 '강남이랑': 1,
 '갖춰진': 1,
 '같네요': 2,
 '같습니다': 2,
 '같아요': 2,
 '같은': 5,
 '같이': 2,
 '개선되어야': 2,
 '개인의': 2,
 '거리': 1,
 '거리로': 1,
 '거리와': 1,
 '거부감이': 1,
 '거실': 1,


### 시각화

In [2]:
# 연설문 1
icon = Image.open(icon_path).convert("RGBA")
mask = Image.new("RGB", icon.size, (255,255,255))
mask.paste(icon,icon)
mask = np.array(mask)

wc = WordCloud(font_path=font_path, background_color="white", max_words=2000, mask=mask,
               max_font_size=300, random_state=42).generate_from_frequencies(tf_word_dict1)

plt.figure(figsize=(30, 90))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()
wc.to_file("house.png") # 그림파일로 추출하기

NameError: name 'Image' is not defined